In [1]:
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore

import numpy as np
from datetime import datetime, time

## Firebase Setup
cred = credentials.Certificate("flutter-testing-c5316-firebase-adminsdk-7os09-7ffa45e8a4.json")
app = firebase_admin.initialize_app(cred)
firestore_client = firestore.client()

## Functions
def in_between(now, start, end):
    if start <= end:
        return start <= now < end
    else: # over midnight e.g., 23:30-04:15
        return start <= now or now < end

def gkernel(x, b, x_data, y_data):
    xx = x - x_data
    K = (1/((np.sqrt(2*np.pi))))*np.exp(-0.5 *(xx/b)**2)
    while np.all(K==0):
        b = b*10
        K = (1/((np.sqrt(2*np.pi))))*np.exp(-0.5 *(xx/b)**2)
    Ksum = np.sum(K)
    weight = K/Ksum
    yk = sum(weight*y_data)
    xkyk = np.array([[x], [yk]])
    return xkyk

# Read data_train
train_set = np.genfromtxt('data_oct.csv', skip_header=1, delimiter=',')

# Get speed data (kph) for regular, AM peak, PM peak
speed_regular = train_set[:,0]
speed_AMpeak = train_set[:,3]
speed_PMpeak = train_set[:,5]

speed_regular = speed_regular[~np.isnan(speed_regular)]
speed_AMpeak = speed_AMpeak[~np.isnan(speed_AMpeak)]
speed_PMpeak = speed_PMpeak[~np.isnan(speed_PMpeak)]

# Get distance data (km) for regular, AM peak, PM peak
# x: input parameter
dist_regular = train_set[:,2]
dist_AMpeak = train_set[:,4]
dist_PMpeak = train_set[:,6]

dist_regular = dist_regular[~np.isnan(dist_regular)]
dist_AMpeak = dist_AMpeak[~np.isnan(dist_AMpeak)]
dist_PMpeak = dist_PMpeak[~np.isnan(dist_PMpeak)]

# Calculate travel time data (minutes) for regular, AM peak, PM peak
# y: ground truth
time_regular = dist_regular / speed_regular 
time_AMpeak = dist_AMpeak / speed_AMpeak
time_PMpeak = dist_PMpeak / speed_PMpeak

now = datetime.now().time()

if in_between(now, time(6), time(9)):
    period = 'AM peak' #Between 6 to 9AM
    b = 500
    x = dist_AMpeak
    y = time_AMpeak
elif in_between(now, time(17), time(21)): 
    period = 'PM peak' #Between 5 to 9PM
    b = 500
    x = dist_PMpeak
    y = time_PMpeak
else:
    period = 'Regular' #Outside AM and PM peak
    b = 100
    x = dist_regular
    y = time_regular

In [2]:
## Update once

buses = firestore_client.collection("buses")

for bus in buses.stream():
    eta_ref = buses.document(bus.id).collection("eta")
    
    for doc in eta_ref.stream():
        distanceEndBus = doc.to_dict()["distanceEndBus"] * 1000
        distanceStartBus = doc.to_dict()["distanceStartBus"] * 1000

        etaEndBus = gkernel(distanceEndBus, b, x, y)[1][0]
        etaStartBus = gkernel(distanceStartBus, b, x, y)[1][0]

        eta_doc = eta_ref.document(doc.id)
        eta_doc.update({"etaEndBus": etaEndBus})
        eta_doc.update({"etaStartBus": etaStartBus})
        
        print(f'Updated ETA of {bus.id}: {doc.id}')

In [ ]:
## Update continuously
import time
buses = firestore_client.collection("buses")

while(True):
    for bus in buses.stream():
        eta_ref = buses.document(bus.id).collection("eta")

        for doc in eta_ref.stream():
            distanceEndBus = doc.to_dict()["distanceEndBus"] * 1000
            distanceStartBus = doc.to_dict()["distanceStartBus"] * 1000

            etaEndBus = gkernel(distanceEndBus, b, x, y)[1][0]
            etaStartBus = gkernel(distanceStartBus, b, x, y)[1][0]

            eta_doc = eta_ref.document(doc.id)
            eta_doc.update({"etaEndBus": etaEndBus})
            eta_doc.update({"etaStartBus": etaStartBus})
            
            print(f'Updated ETA of {bus.id}: {doc.id}')
            
    time.sleep(60)